# US Invitational Young Physicists Tournament 2024 - The Surface of the Moon

## Imports

In [ ]:
import os
try:
    import matplotlib.pyplot as plt
    import cv2
    import glob
    import os
    import numpy as np
    from mpl_toolkits.mplot3d import Axes3D
    import plotly.graph_objects as go
    from numpy import loadtxt
    from coordinates import *
    from datetime import datetime, timedelta
    from angle import get_ra_dec
    from astropy.coordinates import SkyCoord
    import astropy.units as u
except ImportError:
    os.system('pip install matplotlib opencv-python mpld3 glob2 numpy plotly astropy')
    import matplotlib.pyplot as plt
    import cv2
    import glob
    import os
    import numpy as np
    from mpl_toolkits.mplot3d import Axes3D
    import plotly.graph_objects as go
    from numpy import loadtxt
    from coordinates import *
    from datetime import datetime, timedelta
    from angle import get_ra_dec
    from astropy.coordinates import SkyCoord
    import astropy.units as u

# Make figure resolution higher
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

## Creating Light Source Direction Vectors for Photometric Stereo

In [ ]:
start_date = datetime(2023, 10, 23)
end_date = datetime(2023, 11, 6)

current_date = start_date
date_list = []

while current_date <= end_date:
    date_list.append(current_date.strftime("%m-%d-%Y") + "_22.55.00")
    current_date += timedelta(days=1)

print(date_list)

def ra_dec_to_spherical(ra, dec):
    coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
    theta = coord.ra.degree
    phi = 90 - coord.dec.degree
    return theta, phi

SE = 150 * (10 ** 6) # km
ME = 384400 # km

file = open("light_directions_copernicus.txt", "w")

for date in date_list:
    ra1, dec1, ra2, dec2 = get_ra_dec(date)
    theta1, phi1 = ra_dec_to_spherical(ra1, dec1)
    theta2, phi2 = ra_dec_to_spherical(ra2, dec2)
    pos1 = SphericalCoordinate(ME, theta1, phi1).to_rectangular()
    pos2 = SphericalCoordinate(SE, theta2, phi2).to_rectangular()
    pos = RectangularCoordinate(pos2.x - pos1.x, pos2.y - pos1.y, pos2.z - pos1.z)
    pos.normalize()
    file.write(f"{pos.x} {pos.y} {pos.z}\n")
    print(f"{pos.x} {pos.y} {pos.z}")
file.close()

In [ ]:
start_date = datetime(2023, 10, 17)
end_date = datetime(2023, 11, 9)

current_date = start_date
date_list = []

while current_date <= end_date:
    date_list.append(current_date.strftime("%m-%d-%Y") + "_22.55.00")
    current_date += timedelta(days=1)

print(date_list)

def ra_dec_to_spherical(ra, dec):
    coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg))
    theta = coord.ra.degree
    phi = 90 - coord.dec.degree
    return theta, phi

SE = 150 * (10 ** 6) # km
ME = 384400 # km

file = open("light_directions_copernicus.txt", "w")

for date in date_list:
    ra1, dec1, ra2, dec2 = get_ra_dec(date)
    theta1, phi1 = ra_dec_to_spherical(ra1, dec1)
    theta2, phi2 = ra_dec_to_spherical(ra2, dec2)
    pos1 = SphericalCoordinate(ME, theta1, phi1).to_rectangular()
    pos2 = SphericalCoordinate(SE, theta2, phi2).to_rectangular()
    pos = RectangularCoordinate(pos2.x - pos1.x, pos2.y - pos1.y, pos2.z - pos1.z)
    pos.normalize()
    file.write(f"{pos.x} {pos.y} {pos.z}\n")
    print(f"{pos.x} {pos.y} {pos.z}")
file.close()

## Algorithm for Photometric Stereo

In [ ]:
def imread(file_name, intensities=None, normalize=False, flag=-1, scale=1.0):
    img_names = glob.glob(file_name) # Get all of a certain file type that matches file_name
    img_list = []
    gray_img_list = []
    if len(img_names) == 0:
        print("[No {} images]".format(file_name))
        exit()
    img_names = sorted(img_names)
    for i, path in enumerate(img_names):
        im = cv2.imread(path, flag)
        img = cv2.resize(im, None, fx=scale, fy=scale) # Read in image
        # Normalize if needed (in this case since it's images of the moon it's almost grayscale so it doesn't make much of a difference anyway)
        if np.any(intensities):
            img = img / intensities[i]
        elif normalize:
            img = img / 255.0
        # img = img.reshape((-1, 1)).squeeze(1)
        img_list.append(img)
        gray_img_list.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    return img_list, gray_img_list

L_direction = loadtxt(f"./light_directions_copernicus.txt") # Load in the light directions

imgs, gray_imgs = imread(f"../../Pictures/Copernicus Crater/Copernicus Cropped/*_resized.png", intensities=None, normalize=False)

imgs = np.array(imgs, dtype=np.uint8)
gray_imgs = np.array(gray_imgs, dtype=np.uint8)
imgs = imgs[:, :, :, 0:3] # why did it come in with 4 channels? I don't know but this fixes it

def photometric_stereo(imgs, L_list, gray_imgs_list):
    """
    Perform photometric stereo using multiple images of an object captured under different light sources.

    Parameters
    ----------
    imgs: np.ndarray
        Numpy array containing all the images of the object taken with different light sources.
    L_list: list
        List of light source directions (vectors with 3 positional elements x, y, z) for each image.

    Returns
    -------
    img_normal: np.ndarray
        Computed surface normals of the object.
    img_albedo: np.ndarray
        Computed albedos (reflectance) of the object.
    img_normal_rgb: np.ndarray
        RGB representation of the surface normals for visualization.
    """

    # Convert L_list to a numpy array for easier manipulation
    count = 1
    L = np.array(L_list)
    # Transpose the array
    Lt = L.T

    # Get the height and width of the first image in the set
    h, w = imgs[0].shape[:2]

    # Initialize arrays for storing computed values
    img_normal = np.zeros((h, w, 3))
    img_albedo = np.zeros((h, w, 3))
    I = np.zeros((len(L_list), 3))
    I_gray = np.zeros(len(L_list))
    normals = np.zeros((h, w, 3))

    # Iterate over each pixel in the images
    for x in range(w):
        for y in range(h):
            # Collect intensity values for each image at the current pixel
            for i in range(len(imgs)):
                I[i] = imgs[i][y][x]
                I_gray[i] = gray_imgs_list[i][y][x]

            # Compute surface normals and albedos using photometric stereo equations
            tmp1 = np.linalg.inv(np.dot(Lt, L))
            tmp2 = np.dot(Lt, I)
            N = np.dot(tmp1, tmp2).T
            rho = np.linalg.norm(N, axis=1)
            img_albedo[y][x] = rho

            # Convert surface normals to grayscale and normalize
            N_gray = N[0] * 0.2989 + N[1] * 0.5870 + N[2] * 0.1140
            Nnorm = np.linalg.norm(N)
            if Nnorm == 0:
                continue
            img_normal[y][x] = N_gray / Nnorm
            normals[y][x] = np.dot(np.linalg.inv(np.dot(Lt, L)), np.dot(Lt, I_gray)).T
            count += 1

    # Convert the computed values for visualization
    img_normal_rgb = ((img_normal * 0.5 + 0.5) * 255).astype(np.uint8)
    img_albedo = (img_albedo / np.max(img_albedo) * 255).astype(np.uint8)
    img_normal_rgb_2 = ((normals * 0.5 + 0.5) * 255).astype(np.uint8)

    return img_normal, img_albedo, img_normal_rgb, normals, img_normal_rgb_2

img_normal, img_albedo, img_normal_rgb, normals, img_normal_rgb_2 = photometric_stereo(imgs, L_direction, gray_imgs)
np.save(f"./output/normals_copernicus.npy", normals)
np.save(f"./output/albedos_copernicus.npy", img_albedo)
plt.figure()
plt.subplot(1,3,1)
example_img_index = 4
img = (imgs[example_img_index] - np.min(imgs[example_img_index])) / (np.max(imgs[example_img_index]) - np.min(imgs[example_img_index]))  #normalizing image between 0-1 for visualization
plt.imshow(img)
plt.title('Example Full Moon')
plt.xticks([])
plt.yticks([])
#plt.plot([1,2,3,4])
plt.subplot(1,3,2)
#plt.plot([1,2,3,4])
plt.imshow(img_albedo)
plt.title('Image Albedo')
plt.xticks([])
plt.yticks([])
plt.subplot(1,3,3)
#plt.plot([1,2,3,4])
plt.imshow(img_normal_rgb)
plt.title('Surface Normals')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
dataPath = "test_data/"
object = []
for obj in (os.listdir(dataPath)):
    object.append(obj)

L_direction = loadtxt(f"./light_directions.txt") # Load in the light directions

imgs = imread(f"./test_data_libration/*_resized.jpg", intensities=None, normalize=False)